In [1]:
import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
import pandas as pd
from random import shuffle


## Multiclass classifier decision tree using ID3 algorithm

In [2]:
feature = []

In [3]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
#   transfer the data metrix to np array in float type.
    a = np.array(matrix)
    a = a.astype(float)
    #print("normalizing the entire dataset:")   
    #print("Before normalizing")
    #print(a)
#   apply the normalization along the 0 axis of a using the formula: (x - x_min)/(x_max - x_min)
    X = (a - a.min(axis=0))/(a.max(axis=0)-a.min(axis=0))
    #print("After normalizing")
    return X

In [4]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
    data = pd.read_csv(file)
    X = data.to_numpy()
    return (X)

F = load_csv("iris.csv")


In [5]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
    np.random.shuffle(ar)
    return ar


In [6]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
    #print(X.shape[0])
#     store the label X[:,-1] to Y
    Y = X[:,-1]
#     reshape y to (Y's length, 1)
    Y = Y.reshape(len(Y),1)

#     store it to j
    j = Y

    #print("J is",j)
#     create the new_X which exclude the label X[:,:-1]
    new_X = X[:,:-1]

#     normalize the data step
#     using our implemented function normalize()
    normalized_X = normalize(new_X)

#     add the label back to the normiazlied X
#     using np.concatenate along axis=1
    X = np.concatenate((normalized_X,Y),axis=1)

#     store the size of rows of the normalized X with labels
    X_rowsize = X.shape[0]

#     use the 10% of the data to be the test set.
#     store the number of testing data

    num_testing_data = math.floor(X_rowsize * 0.1)

#     set the starting idex to be 0
    start = 0

#     set the ending index to be the number of testing data
    end = num_testing_data

#     create a list that store all features of the testing data
    test_features = []

#     create a list that store all labels of the testing data
    test_labels = []

#     create a list that store all features of the training data
    train_features = []

#     create a list that store all labels of the training data
    train_labels = []

#     10-fold cross-validation:
    for i in range(10):
#         store the test set for corss-validation using X[start:end,:]
      test_set = X[start:end,:]
    
#         get training data before the testing data X[:start, :]
      train_btest = X[:start,:]

#         get training data after the testing data X[:start, :]
      train_atest = X[end:,:]

#         form the new training set using np.concatenate
      train_set = np.concatenate((train_btest,train_atest),axis=0)

#         get the testing set labels
      test_set_labels = test_set[:,-1]

#         flattent the labels
      test_set_labels.flatten('C')

#         get the training set labels
      train_set_labels = train_set[:,-1]

#         flattent the labels
      train_set_labels.flatten('C')

#         create the test set exclude the labels
      test_set = test_set[:,:-1]


#         same for the training set
      train_set = train_set[:,:-1]


#         append test data of this fold to the list
      test_features.append(test_set)

#         append test lables of this fold to the list
      test_labels.append(test_set_labels)

#         do the same for the training set
      train_features.append(train_set)
      train_labels.append(train_set_labels)


#         update the index pointer
      start=end
      end=end+num_testing_data

#     return the fold list that contain data and label for both training and testing set.
    return train_features,train_labels,test_features,test_labels


In [7]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
#     init a dict for attributes
    attribute_dict = {}
#     init feature list.
    f_list = []

#     for each feature in the dataset
    for i in range(X.shape[1]):
#       store the featur index
        feature_index = i

#     find all the value correspond to this feature
        vals = X[:,i]

#     init an attribute list
        attributes = []

#     init the counter to 0
        cntr = 0

#         for each value in the "all the value correspond to this feature"
        for x in range(len(vals)):
            value = vals[x]
#             init a empty list that store the attribute value
            a_value = []
#             append the this value to the list
            a_value.append(value)

#             append the label of this value to the list
            a_value.append(y[x])

#             append this list to the attribute list.
            attributes.append(a_value)

#             increase the counter
            cntr+=1

#         add this attribute list to the dict according to the feature index
        attribute_dict[feature_index] = attributes

#         append the feature indx to the feature list.
        f_list.append(feature_index)

#     return the dict and feature list.

    return attribute_dict,f_list

In [8]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
#     init a class frequence dict
    c_freq = {}
    
#     init the attribute entropy to 0
    att_entropy = 0

#     for each label in y:
    for i in y:
#         this is label is already in the dict, we increase its feq
        if i in c_freq:
                c_freq[i]+=1
        

#          else, we set the freq to 1
        else:
                c_freq[i] = 1

#     calculate the cumulate entropy using the formula.
    t_count = 0
    for i in c_freq: 
        t_count = t_count + c_freq[i] 
      
    for x in c_freq:
        p = c_freq[x]/t_count
        log_p = math.log(p)
        att_entropy+= -(p*log_p)

    return att_entropy


In [9]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,thea and leaf.
    def __init__(self, val, lchild, rchild,thea,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = thea
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self):
        return self.leaf

#     method to return threshold value
    def ret_thetha(self):
        return self.thetha
    
#     method return root value
    def ret_root_value(self):
        return self.root_value
    
#     method return left tree
    def ret_llist(self):
        return self.root_left

#     method return right tree
    def ret_rlist(self):
        return self.root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [10]:
#Decision tree object
class DecisionTree(object):
#     init a variable called fea_list
    fea_list = []

#     init the Dtree by setting the root node to None
    def __init__(self):
        self.root_node = None

    #method to return the major class value using Counter() and .most_common()
    def cal_major_class_values(self,class_values):
        most_com_val =  Counter(class_values).most_common(1)[0][0] 
        return most_com_val

    #method to calculate best threshold value for each feature
    def cal_best_theta_value(self,ke,attri_list):
#         init a list for data
        d_list = []

#         init a list for class labes
        c_labels = []

#         for each attribute in the attri_list
        for i in attri_list:
#             append the data
            d_list.append(i[0])

#             append the feature value.
            c_labels.append(i[1])

#         calculate the entropy of those feaure values
            entropy_val = entropy(c_labels)
    
#         init the max info gain = 0
            max_info_gain = 0

#         init theta=0
            theta = 0

#         init a list that store the best index on the left
            best_index_left = []

#         init a list that store the best index on the right
            best_index_right = []

#         init a list that store class labels after split
            c_labels_after_split = []

#         sort the data
            d_list.sort()

#         for each index of data:
        for i in range(len(d_list)):
#             calculate the current theta using data[i]+data[i+1])/ 2
            if (i<(len(d_list)-1)):
                    current_theta = (d_list[i]+d_list[i+1])/2
#             init a list that store index that less than theta
            less_than_theta_index = []
#             init a list that store value that less than theta
            less_than_theta_value = []
    
#             init a list that store index that greater than theta
            greater_than_theta_index = []

#             init a list that store value that less than theta
            greater_than_theta_value = []

#             init the counter to 0
            counter = 0 

#             for each index and value in attri_list
            for c,j in enumerate(attri_list):
#                 if value less or equal than the current theta:
#                     update the "less" list of index and value
                if (j[0]<=current_theta):
                    less_than_theta_index.append(c)
                    less_than_theta_value.append(j[1])

#                 else:
#                     update the "greater" list of index and value
                else:
                    greater_than_theta_index.append(c)
                    greater_than_theta_value.append(j[1])


#             calculate the entropy of the "less" list
            el = [c_labels[v] for v in less_than_theta_index]
            entropy_less_list = entropy(el)
#             calculate the entropy of the "greater" list
            eg = [c_labels[w] for w in greater_than_theta_index]
            entropy_greater_list = entropy(eg)
#             calculate the info gain using the formular.
            len_eleft = len(el)
            len_eright = len(eg)
            current_infogain = entropy_val - (((len_eleft/len(c_labels))*entropy_less_list)+((len_eright/len(c_labels))*entropy_greater_list))

#             if current info gain > max info gan

#                 update the info gain, 
#                     the theta, the best index list of right, 
#                         the best index list of left and class_labels_list_after_split

            if current_infogain > max_info_gain:
                    max_info_gain = current_infogain
                    theta = current_theta
                    best_index_left = less_than_theta_index
                    best_index_right = greater_than_theta_index
                    l_and_r_labels = el + eg
                    class_labels_after_split = l_and_r_labels


#         return the max info gain, theata,the best left list,the best right list and class label after split
        return max_info_gain,theta,best_index_left,best_index_right,class_labels_after_split

    #method to select the best feature out of all the features.
    def best_feature(self,dict_rep):
#         set key value to none
        key_value = None
    
#         set best info gain to -1
        best_info_gain = -1

#         set best theta to 0
        best_theta = 0

#         set best left list to empty
        best_left_list = []

#         set best right list to empty
        best_right_list = []

#         set best class labels after split to empty
        best_class_labels = []

#         init a result list
        result_list = []

#         for each key in dict_rep:
        for key in dict_rep.keys():
#             using cal_best_theta_value() and store all returned values
            info_gain,theta,index_left,index_right,class_labels_after_split = self.cal_best_theta_value(key,dict_rep[key])

#             if info gain is greater than best info gain:
#                 update info gain, theth, key value,
#                     left list, right list, class labels after split

            if info_gain>best_info_gain:
                key_value = key
                best_info_gain = info_gain
                best_theta = theta
                best_left_list = index_left
                best_right_list = index_right
                best_class_labels = class_labels_after_split

#         append the key value to the retrun list
        result_list.append(key_value)

#         append the theta value to the retrun list
        result_list.append(best_theta)

#         append the left list to the retrun list
        result_list.append(best_left_list)

#         append the right list to the retrun list
        result_list.append(best_right_list)

#         append the class labels to the retrun list
        result_list.append(best_class_labels)

#         return the list.
        return result_list

    def get_remainder_dict(self,dict_of_everything,index_split):
        global fea_list
#         init a split dict
        split_dictionary = {}

#         for each key "ke" in dict_of_everything:
        for ke in dict_of_everything.keys():
#             init a value list
            value_list = []

#             init a modified list
            mod_list = []

#             get the corresponding values of the key"ke" 
            val_in_key = dict_of_everything[ke]

#             for each value and its corresponding index of the key"ke" 
            for index in range(len(val_in_key)):
#                 if index is not in the index_split:
                if index not in index_split:
#                     append it to the modified list and value list
                    mod_list.append(val_in_key[index])
                    value_list.append(val_in_key[index][1])
#             add this modified list to the dict
            split_dictionary[ke]= mod_list
#         return the splited dict and val list
        return split_dictionary,value_list

    #method to create decision tree
    
    
    def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
        global fea_list
        #if all the class labels are same, then we are set
        if len(set(class_val)) ==1:
            #print("Leaf node for set class is",class_val[0],len(class_val))
            return Node(class_val[0],None,None,None,True)
        #if the no class vales are less than threshold, we assign the class with max values as the class label    
        elif len(class_val) < eta_min_val:
            class_label = self.cal_major_class_values(class_val)
            return Node(class_label,None,None,None,True)
#         else:
        else:
#             using the best_feature to get best feature list
            best_feature_list = self.best_feature(dict_of_everything)

#             store the node name, theta,left split, right split and class labes
            node_name = best_feature_list[0]
            theta = best_feature_list[1]
            left_split = best_feature_list[2]
            right_split = best_feature_list[3]
            labels = best_feature_list[4]

#             call get_remainder_dict to get left tree data
            l_subtree_dict,l_subtree_class = self.get_remainder_dict(dict_of_everything,left_split)

#             call get_remainder_dict to get right tree data
            r_subtree_dict,r_subtree_class = self.get_remainder_dict(dict_of_everything,right_split)

#             call create_decision_tree to get left tree based on the left tree data
            l_subtree_node = self.create_decision_tree(l_subtree_dict,l_subtree_class,eta_min_val)

#             call create_decision_tree to get right tree based on therightleft tree data
            r_subtree_node = self.create_decision_tree(r_subtree_dict,r_subtree_class,eta_min_val)

#             set the root node
            root_node =  Node(node_name,r_subtree_node,l_subtree_node,theta,False)

#             return root node
            return root_node
            
    #fit the decisin tree
    def fit(self, dict_of_everything,cl_val,features,eta_min_val):
#         set the fea_list the value of features
        global fea_list
        fea_list = features

#         set the root node using the function create_decision_tree()
        self.root_node  = self.create_decision_tree(dict_of_everything,cl_val,eta_min_val)
    
        return self.root_node 

    def classify(self,row,root):
#         init the test dict
#         add row to the dict
        current_node = root

#         while the current node is not leaf:
        while not current_node.leaf:
#             implement the case whether the current shoud go to the left
            if row[current_node.root_value]<current_node.theta:
                current_node  = current_node.root_left
 #             implement the case whether the current shoud go to the right
            else:
                current_node = current_node.root_right
            
            result = current_node.root_value        
#         return the calss of the current node

        return result
        
    #method to the labels for the test data
    def predict(self, X, root):
#         predict using the classify()
        results = []
        for row in X:
            r_result = self.classify(row,root)
            results.append(r_result)
        return results



In [11]:
#calculating the predicited accuracy
def accuracy_for_predicted_values(test_class_names,l):
#     init true and false count to 0
    true_count = 0 
    false_count = 0 
#     for each prediction,if predict is correct then, true++ else, false++
    for i in range(len(test_class_names)):
        if test_class_names[i] == l[i]:
            true_count = true_count+1 
        else:
            false_count = false_count+1
    acc = (true_count)/(true_count+false_count)
        
#     return the acc
    return acc


In [12]:
def main(num_arr, eta_min):
    eta_min_val = round(eta_min*num_arr.shape[0])
    
    #randomly shuffle the array so that we can divide the data into test/training
    num_arr = random_numpy_array(num_arr)
    
    #divide data into test labels,test features,training labels, training features
    train_features,train_labels,testing_features,test_labels = generate_set(num_arr) 

    #init cumulate acc to 0
    cumulate_acc = 0

    #ten fold iteration 
    for i in range(10):
        #build a dictionary with class labels and respective features values belonging to that class
        attribute_dict,feature_list = build_dict_of_attributes_with_class_values(train_features[i],train_labels[i])
        
        #instantiate decision tree instance
        DTree = DecisionTree()
        
        # build the decision tree model.
        DTree.fit(attribute_dict,train_labels[i],feature_list,eta_min_val)
        
        #predict the class labels for test features
        predicted_labels = DTree.predict(testing_features[i],DTree.root_node)
        
        #calculate the accuracy for the predicted values    
        accu = accuracy_for_predicted_values(predicted_labels,test_labels[i])
        
#         add acc to cumulate acc
        cumulate_acc+=accu
        print("Accuracy is ",accu)
    print("Accuracy across 10-cross validation for",eta_min,"is",float(cumulate_acc)/10)


In [13]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.8571428571428571
Accuracy is  1.0
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy across 10-cross validation for 0.05 is 0.9714285714285715
Accuracy is  0.8571428571428571
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  0.9285714285714286
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  0.9285714285714286
Accuracy across 10-cross validation for 0.1 is 0.9500000000000002
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.9285714285714286
Accuracy is  0.8571428571428571
Accuracy is  0.9285714285714286
Accuracy is  0.9285714285714286
Accuracy is  1.0
Accuracy is  1.0
Accuracy across 10-cross validation for 0.15 is 0.9571428571428573
Accuracy is  0.9285714285714286
Accuracy is  0.9285714285714286
Accuracy is  0.9

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "spambase.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)